In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import pickle

In [2]:
N_DAYS = 100
N_FAMILIES = 5000
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125
N_OCCUPANCY = 176

data = pd.read_csv('family_data.csv', index_col='family_id')

FAMILY_SIZE = data.n_people.values

CHOICES = data.drop('n_people', axis=1).values -1
# with open ('santa-workshop-tour-2019/Choice_Table.pkl', 'rb') as fp: CHOICES = pickle.load(fp)

In [3]:
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    elif choice == 1:
        penalty = 50
    elif choice == 2:
        penalty = 50 + 9 * n
    elif choice == 3:
        penalty = 100 + 9 * n
    elif choice == 4:
        penalty = 200 + 9 * n
    elif choice == 5:
        penalty = 200 + 18 * n
    elif choice == 6:
        penalty = 300 + 18 * n
    elif choice == 7:
        penalty = 300 + 36 * n
    elif choice == 8:
        penalty = 400 + 36 * n
    elif choice == 9:
        penalty = 500 + 36 * n + 199 * n
    else:
        penalty = 500 + 36 * n + 398 * n
    return penalty


def GetPreferenceCostMatrix(data):
    cost_matrix = np.zeros((N_FAMILIES, N_DAYS), dtype=np.int32)
    for i in range(N_FAMILIES):
        desired = data.values[i, :-1]
        cost_matrix[i, :] = get_penalty(FAMILY_SIZE[i], 10)
        for j, day in enumerate(desired):
            cost_matrix[i, day-1] = get_penalty(FAMILY_SIZE[i], j)
    return cost_matrix


def GetAccountingCostMatrix():
    ac = np.zeros((N_OCCUPANCY, N_OCCUPANCY), dtype=np.float32)
    for i,n in enumerate(range(125,301)):
        for j,n_p1 in enumerate(range(125,301)):
            diff = abs(n - n_p1)
            ac[i, j] = max(0, (n - 125) / 400 * n**(0.5 + diff / 50.0))
    return ac

In [4]:
PREFCOST = GetPreferenceCostMatrix(data)
ACCOST = GetAccountingCostMatrix()
PREFCOST.shape

(5000, 100)

In [6]:
# Create a new model
model = gp.Model("santa")

disabled_bad_choice = [[1 if d in row[:5] else 0 for d in range(100) ] for r, row in enumerate(CHOICES)]
x_lb  = [[0 for d in range(100) ] for r, row in enumerate(CHOICES)]

#Preference Cost VARS
x = model.addVars(N_FAMILIES, N_DAYS, ub=1.0, vtype=GRB.BINARY, name='x')
# Accounting Cost VARS
y = model.addVars(N_DAYS,N_OCCUPANCY,N_OCCUPANCY, ub=1.0, vtype=GRB.BINARY, name='y')


#Preference cost
preference_cost = [PREFCOST[f,d]*x[f,d] for f in range(N_FAMILIES) for d in range(N_DAYS)]
preference_cost = sum(preference_cost)


occupancy={}
for d in range(N_DAYS):
    occupancy[d] = sum([x[f,d]*FAMILY_SIZE[f] for f in range(N_FAMILIES)])
occupancy[d+1] = occupancy[d]


for f in range(N_FAMILIES):
    model.addConstr(sum([x[f,d] for d in range(N_DAYS)]) == 1)

    
for d in range(N_DAYS):
    model.addConstr(occupancy[d] >= 125)
    model.addConstr(occupancy[d] <= 300)

    
count= np.arange(125,301)
for d in range(N_DAYS):
    y_sum_u = sum( y[d,u,v]*count[u] for u in range(N_OCCUPANCY) for v in range(N_OCCUPANCY) )
    y_sum_v = sum( y[d,u,v]*count[v] for u in range(N_OCCUPANCY) for v in range(N_OCCUPANCY) )
    model.addConstr(y_sum_u == occupancy[d])
    model.addConstr(y_sum_v == occupancy[d+1])
    
    y_sum = sum( y[d,u,v] for v in range(N_OCCUPANCY) for u in range(N_OCCUPANCY) )
    model.addConstr(y_sum == 1)

for d in range(N_DAYS -1):
    for t in range(N_OCCUPANCY):
        y_sum_u = sum( y[d,u,t] for u in range(N_OCCUPANCY) )
        y_sum_v = sum( y[d+1,t,v] for v in range(N_OCCUPANCY) )
        model.addConstr(y_sum_u == y_sum_v)


## Callbacks ##
def simple_mst_writer(model, mstfilename, nodecnt, obj):
    mstfile = open(mstfilename, 'w')
    varlist = model.getVars()
    soln = model.cbGetSolution(varlist) #  cbGetSolution
    mstfile.write('# MIP start from soln at node %d obj %e\n' % (nodecnt, obj))
    for var, soln in zip(varlist, soln):
        mstfile.write('%s %.3e\n' % (var.VarName, soln))
    mstfile.close()

def save_output(model,score):
    days = []
    for v in model.getVars()[:500000]:
        if int(v.X)==1:
            f, d = v.varName.replace("x[","").replace("]","").split(',')
            days.append(int(d)+1)
    df = pd.DataFrame(days)
    df.reset_index(level=0, inplace=True)
    df.columns = ['family_id' , 'assigned_day']
    df.to_csv(f'{score}.csv', index=False)

def mycallback(model, where):
    if where == GRB.callback.MIPSOL: # MIPSOL
        obj = model.cbGet(GRB.callback.MIPSOL_OBJ)
        nodecnt = int(model.cbGet(GRB.callback.MIPSOL_NODCNT))
        print('Found solution at node', nodecnt, 'obj', obj)
        save_output(model, obj)
#         simple_mst_writer(model, 'sol.mst', nodecnt, obj)

#Accounting Cost
acc_cost = [ACCOST[u,v]*y[d,u,v] for u in range(N_OCCUPANCY) for v in range(N_OCCUPANCY) for d in range(N_DAYS)]
acc_cost = sum(acc_cost)


### Init Solution ###
init = pd.read_csv('69866.csv')
init_day = init[['family_id', 'assigned_day']].values

for f in range(N_FAMILIES):
    for d in range(N_DAYS):
        x[f, d].start =  False
    day = init_day[f, 1]
    x[f, day-1].start = True
model.update()

model.setParam('Seed', 2090) 
# model.setParam('TimeLimit',5*60)

# Model Param
model.setParam('MIPGap', 0)
model.setObjective(preference_cost + acc_cost, GRB.MINIMIZE)
print('all ready') 

Changed value of parameter Seed to 2090
   Prev: 0  Min: 0  Max: 2000000000  Default: 0
Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
all ready


In [61]:
## Callbacks ##
def simple_mst_writer(model, mstfilename, nodecnt, obj):
    mstfile = open(mstfilename, 'w')
    varlist = model.getVars()
    soln = model.cbGetSolution(varlist) #  cbGetSolution
    mstfile.write('# MIP start from soln at node %d obj %e\n' % (nodecnt, obj))
    for var, soln in zip(varlist, soln):
        mstfile.write('%s %.3e\n' % (var.VarName, soln))
    mstfile.close()

def save_output(model,score):
    days = []
    for v in model.getVars()[:500000]:
        if int(v.X)==1:
            f, d = v.varName.replace("x[","").replace("]","").split(',')
            days.append(int(d)+1)
    df = pd.DataFrame(days)
    df.reset_index(level=0, inplace=True)
    df.columns = ['family_id' , 'assigned_day']
    df.to_csv(f'{score}.csv', index=False)

def mycallback(model, where):
    try:
        if where == GRB.callback.MIPSOL: # MIPSOL
            obj = model.cbGet(GRB.callback.MIPSOL_OBJ)
            nodecnt = int(model.cbGet(GRB.callback.MIPSOL_NODCNT))
            print('Found solution at node', nodecnt, 'obj', obj)
            simple_mst_writer(model, f'{obj}.mst', nodecnt, obj)
    except KeyboardInterrupt:
        model.terminate()

model.setParam('TimeLimit',99999999*60)
model.setParam('Seed', 2049)
model.setParam('MIPGap', 0)

Parameter TimeLimit unchanged
   Value: 5999999940.0  Min: 0.0  Max: inf  Default: inf
Parameter Seed unchanged
   Value: 2049  Min: 0  Max: 2000000000  Default: 0
Parameter MIPGap unchanged
   Value: 0.0  Min: 0.0  Max: inf  Default: 0.0001


In [60]:
init = pd.read_csv('68928.711634.csv')
init_day = init[['family_id', 'assigned_day']].values

for f in range(N_FAMILIES):
    for d in range(N_DAYS):
        x[f, d].start =  False
    day = init_day[f, 1]
    x[f, day-1].start = True
model.update()

In [62]:
try:
    model.optimize(mycallback)

    print('Obj: %g' % model.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))
except AttributeError:
    print('Encountered an attribute error')
except KeyboardInterrupt:
    model.terminate()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 22924 rows, 3597600 columns and 17926048 nonzeros
Model fingerprint: 0x1f3a5103
Variable types: 0 continuous, 3597600 integer (3597600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-02, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Found solution at node 0 obj 68928.71163427457
User MIP start produced solution with objective 68928.7 (53.17s)
Found solution at node 0 obj 68928.71163427457
Loaded user MIP start with objective 68928.7
Found solution at node 0 obj 68890.12205032259
Loaded MIP start from previous solve with objective 68890.1
Processed MIP start in 85.28 seconds

Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 0 rows and 0 columns (presolve time = 10s) ...
Presolve removed 100 rows and 0 col

In [63]:
model.setParam('Seed', 90051)

Changed value of parameter Seed to 90051
   Prev: 2049  Min: 0  Max: 2000000000  Default: 0


In [64]:
try:
    model.optimize(mycallback)

    print('Obj: %g' % model.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))
except AttributeError:
    print('Encountered an attribute error')
except KeyboardInterrupt:
    model.terminate()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 22924 rows, 3597600 columns and 17926048 nonzeros
Model fingerprint: 0x1f3a5103
Variable types: 0 continuous, 3597600 integer (3597600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-02, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolved: 22823 rows, 3565430 columns, 16199387 nonzeros

Continuing optimization...

  5225  5163 68780.1479  724  600 68890.1221 68204.8281  0.99%   831 33069s
  5400  5285 68784.2927  752  632 68890.1221 68204.8281  0.99%   841 33398s
  5544  5393 68795.7687  777  579 68890.1221 68204.8281  0.99%   833 33771s
  5663  5505 68816.2541  802  588 68890.1221 68204.8281  0.99%   832 34181s
  5777  5606 68817.0164  831  582 68890.1221 68204.8281  0.99%   833 34578s
  5902  5639 infeasible  838      68890.1221 

In [48]:
def save_output(model,score):
    days = []
    for v in model.getVars()[:500000]:
        if int(v.X)==1:
            f, d = v.varName.replace("x[","").replace("]","").split(',')
            days.append(int(d)+1)
    df = pd.DataFrame(days)
    df.reset_index(level=0, inplace=True)
    df.columns = ['family_id' , 'assigned_day']
    df.to_csv(f'{score}.csv', index=False)
save_output(model,68928.711634")